In [2]:
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt

In [3]:
lic = pd.read_csv( "https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/licenses_fall2022.csv")

In [4]:
lic

,_id,License Type,Description,License Number,License Status,Business,Title,First Name,Middle,Last Name,...,Specialty/Qualifier,Controlled Substance Schedule,Delegated Controlled Substance Schedule,Ever Disciplined,LastModifiedDate,Case Number,Action,Discipline Start Date,Discipline End Date,Discipline Reason
0,1189509,DETECTIVE BOARD,PERMANENT EMPLOYEE REGISTRATION,129446286,NOT RENEWED,N,NaN,EILEEN,NaN,SANTACRUZ,...,NaN,NaN,NaN,N,03/18/2022,NaN,NaN,NaN,NaN,NaN
1,801037,DETECTIVE BOARD,FIREARM CONTROL CARD,229030294.0,NOT RENEWED,N,NaN,DAGMAR,J,NORDLUND,...,NaN,NaN,NaN,N,08/16/2006,NaN,NaN,NaN,NaN,NaN
2,365129,COSMO,LICENSED COSMETOLOGIST,11053076.0,NOT RENEWED,N,NaN,RADOJE,NaN,ZELENOVIC,...,NaN,NaN,NaN,N,05/26/2006,NaN,NaN,NaN,NaN,NaN
3,595427,COSMO,LICENSED COSMETOLOGIST,11295645.0,ACTIVE,N,NaN,BECKY SUE,L,BURROUGHS,...,NaN,NaN,NaN,N,11/12/2021,NaN,NaN,NaN,NaN,NaN
4,653668,COSMO,LICENSED NAIL TECHNICIAN,169006247,NOT RENEWED,N,NaN,BILL G,L,LETNER,...,NaN,NaN,NaN,N,05/30/2006,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,888281,DETECTIVE BOARD,PERMANENT EMPLOYEE REGISTRATION,129002843.0,NOT RENEWED,N,NaN,JENNIFER,NaN,DARROW,...,NaN,NaN,NaN,N,08/03/2006,NaN,NaN,NaN,NaN,NaN
9996,766623,DETECTIVE BOARD,FIREARM CONTROL CARD,229014180,TERMINATED CARD RETURNED,N,NaN,BRYAN,NaN,WILLIAMS,...,NaN,NaN,NaN,N,08/07/2006,NaN,NaN,NaN,NaN,NaN
9997,399398,COSMO,LICENSED COSMETOLOGIST,11120249,NOT RENEWED,N,NaN,EUGENE,NaN,HENDERSON JR,...,NaN,NaN,NaN,N,05/26/2006,NaN,NaN,NaN,NaN,NaN
9998,486713,COSMO,LICENSED COSMETOLOGIST,11193270,ACTIVE,N,NaN,MAHLON DOUGLAS,NaN,CLIFT,...,NaN,NaN,NaN,N,12/17/2021,NaN,NaN,NaN,NaN,NaN


In [5]:
lic['Ever Disciplined'].unique()

array(['N', 'Y'], dtype=object)

In [6]:
lic.columns

Index(['_id', 'License Type', 'Description', 'License Number',
       'License Status', 'Business', 'Title', 'First Name', 'Middle',
       'Last Name', 'Prefix', 'Suffix', 'Business Name', 'BusinessDBA',
       'Original Issue Date', 'Effective Date', 'Expiration Date', 'City',
       'State', 'Zip', 'County', 'Specialty/Qualifier',
       'Controlled Substance Schedule',
       'Delegated Controlled Substance Schedule', 'Ever Disciplined',
       'LastModifiedDate', 'Case Number', 'Action', 'Discipline Start Date',
       'Discipline End Date', 'Discipline Reason'],
      dtype='object')

In [7]:
lic['License Status'].unique()

array(['NOT RENEWED', 'ACTIVE', 'CANCELLED', 'TERMINATED CARD RETURNED',
       'DECEASED', 'INACTIVE', 'EXPIRED', 'CLOSED',
       'TERMINATED WITHOUT CARD', 'INOPERATIVE',
       'TERMINATED VALID REASON', 'CHANGE OF OWNERSHIP',
       'Non Sufficient Fund Check Terminated'], dtype=object)

In [8]:
lic.dtypes


_id                                          int64
License Type                                object
Description                                 object
License Number                              object
License Status                              object
Business                                    object
Title                                       object
First Name                                  object
Middle                                      object
Last Name                                   object
Prefix                                      object
Suffix                                      object
Business Name                               object
BusinessDBA                                 object
Original Issue Date                         object
Effective Date                              object
Expiration Date                             object
City                                        object
State                                       object
Zip                            

In [9]:
# Parse date column safely
lic['Original Issue Year'] = pd.to_datetime(lic['Original Issue Date'], errors='coerce').dt.year

# Drop rows without a valid issue date
lic = lic.dropna(subset=['Original Issue Year']).copy()

# Define mapping for license status
status_map = {
    'ACTIVE': 'Active',
    'INACTIVE': 'Inactive',
    'INOPERATIVE': 'Inactive',
    'EXPIRED': 'Expired',
    'NOT RENEWED': 'Expired',
    'CANCELLED': 'Terminated',
    'CLOSED': 'Terminated',
    'TERMINATED CARD RETURNED': 'Terminated',
    'TERMINATED WITHOUT CARD': 'Terminated',
    'TERMINATED VALID REASON': 'Terminated',
    'CHANGE OF OWNERSHIP': 'Terminated',
    'Non Sufficient Fund Check Terminated': 'Terminated',
    'DECEASED': 'Deceased'
}

# Apply mapping
lic['Status_Group'] = lic['License Status'].map(status_map)
lic_subset=lic[['Status_Group','Original Issue Year']]


In [10]:
lic_subset['Original Issue Year'].describe()

count    9995.000000
mean     1999.410905
std        14.895539
min      1912.000000
25%      1992.000000
50%      1999.000000
75%      2011.000000
max      2022.000000
Name: Original Issue Year, dtype: float64

In [11]:
from IPython.display import HTML

HTML("""
<style>
.vega-tooltip {
  box-shadow: none !important;
}
</style>
""")

In [12]:
alt.data_transformers.disable_max_rows()
years = sorted(lic['Original Issue Year'].unique())
tick_years = years[::5]

# Create interval brush
brush = alt.selection_interval(encodings=['x'], name='year_brush')
# Line chart for issued licenses per year
issued_chart = (
    alt.Chart(lic_subset)
    .mark_line(point=True)
    .encode(
        x=alt.X('Original Issue Year:O', title='Year of Issue', axis=alt.Axis(values=tick_years, labelAngle=0)),
        y=alt.Y('count():Q', title='Number of Licenses Issued'),
        tooltip=[
            alt.Tooltip('Original Issue Year:O', title='Year'),
            alt.Tooltip('count():Q', title='Issued')
        ]
    )
    .properties(width=750, height=300, title='Issued Licenses per Year')
    .add_params(brush)  # brush for selection
)

status_chart = (
    alt.Chart(lic_subset)
    .mark_bar()
    .encode(
        y=alt.Y('Status_Group:N',
                sort='-x',
                title='License Status'),
        x=alt.X('count():Q',
                title='Count of Licenses'),
        tooltip=[
            alt.Tooltip('Status_Group:N', title='Status'),
            alt.Tooltip('count():Q', title='Count')
        ],
        color=alt.Color('Status_Group:N', legend=None)  # quitar leyenda si no es necesaria
    )
    .transform_filter(brush)
    .properties(width=300, height=250, title='License Status Counts')
)
chart=(issued_chart & status_chart)
chart


alt.VConcatChart(...)

In [13]:
myJekyllDir='/Users/Pc/DAJV-21.github.io/assets/json/'

In [14]:
chart.save(myJekyllDir+'Home5.json')

In [15]:
import os
os.stat(myJekyllDir+'Home5.json').st_size

601253